## 1. Kolla exempelrader

In [110]:
import pandas as pd
from pathlib import Path

df = pd.read_csv("src/taxi_prediction_fullstack_orhan/data/taxi_trip_pricing.csv")  # Csv-filen laddas in i en DataFrame

df.head()  # Visar de första fem raderna i DataFrame:n

,Trip_Distance_km,Time_of_Day,Day_of_Week,Passenger_Count,Traffic_Conditions,Weather,Base_Fare,Per_Km_Rate,Per_Minute_Rate,Trip_Duration_Minutes,Trip_Price
0,19.35,Morning,Weekday,3.0,Low,Clear,3.56,0.80,0.32,53.82,36.2624
1,47.59,Afternoon,Weekday,1.0,High,Clear,NaN,0.62,0.43,40.57,NaN
2,36.87,Evening,Weekend,1.0,High,Clear,2.70,1.21,0.15,37.27,52.9032
3,30.33,Evening,Weekday,4.0,Low,NaN,3.48,0.51,0.15,116.81,36.4698
4,NaN,Evening,Weekday,3.0,High,Clear,2.93,0.63,0.32,22.64,15.6180


## 2. Datamängdens storlek

In [111]:
df.shape # Visar antalet rader och kolumner i DataFrame:n
# (antal_rader, antal_kolumner)

(1000, 11)

## 3. Vilka features som finns

In [112]:
df.columns # Visar kolumnnamnen i DataFrame:n


Index(['Trip_Distance_km', 'Time_of_Day', 'Day_of_Week', 'Passenger_Count',
       'Traffic_Conditions', 'Weather', 'Base_Fare', 'Per_Km_Rate',
       'Per_Minute_Rate', 'Trip_Duration_Minutes', 'Trip_Price'],
      dtype='object')

## 4. Datatyper + nullvärden

In [113]:
df.info() # Visar en sammanfattning av DataFrame:n inklusive datatyper och icke-noll värden


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Trip_Distance_km       950 non-null    float64
 1   Time_of_Day            950 non-null    object 
 2   Day_of_Week            950 non-null    object 
 3   Passenger_Count        950 non-null    float64
 4   Traffic_Conditions     950 non-null    object 
 5   Weather                950 non-null    object 
 6   Base_Fare              950 non-null    float64
 7   Per_Km_Rate            950 non-null    float64
 8   Per_Minute_Rate        950 non-null    float64
 9   Trip_Duration_Minutes  950 non-null    float64
 10  Trip_Price             951 non-null    float64
dtypes: float64(7), object(4)
memory usage: 86.1+ KB


## 5. Exakt var nullvärden finns

In [114]:
df.isna().sum() # Kontroll för saknade värden i varje kolumn


Trip_Distance_km         50
Time_of_Day              50
Day_of_Week              50
Passenger_Count          50
Traffic_Conditions       50
Weather                  50
Base_Fare                50
Per_Km_Rate              50
Per_Minute_Rate          50
Trip_Duration_Minutes    50
Trip_Price               49
dtype: int64

## Kolumnförståelse

- Trip_Distance_km: resans längd i km
- Time_of_Day: Morning / Afternoon / Evening / Night
- Day_of_Week: Weekday eller Weekend
- Passenger_Count: antal passagerare
- Traffic_Conditions: Low / Medium / High
- Weather: Clear / Rain / Snow
- Base_Fare: grundpris
- Per_Km_Rate: pris per km
- Trip_Price: totalpris (label/target)


## Målet är att bygga en modell som kan prediktera Trip_Price baserat på övriga tillgängliga features.

In [115]:
# Ta bort rader där target saknas
df_model = df.dropna(subset=["Trip_Price"])

In [116]:
df_model = df_model.dropna()  # Tar bort alla återstående rader med nullvärden

In [117]:
df_model.isna().sum()  # Kontroll för saknade värden i varje kolumn(efter borttagning)
df_model.shape         # Visar antalet rader och kolumner i den rengjorda DataFrame:n

(562, 11)

In [118]:
x = df_model.drop(columns=["Trip_Price"])  # Funktioner (features)
y = df_model["Trip_Price"]                 # Målvariabel (target)

In [119]:
x.dtypes # Visar datatyperna för varje kolumn i funktionerna
# vi behöver konvertera vissa kolumner till kategoriska datatyper

Trip_Distance_km         float64
Time_of_Day               object
Day_of_Week               object
Passenger_Count          float64
Traffic_Conditions        object
Weather                   object
Base_Fare                float64
Per_Km_Rate              float64
Per_Minute_Rate          float64
Trip_Duration_Minutes    float64
dtype: object

In [120]:
x_encoded = pd.get_dummies(x, drop_first=True)  # One-hot encoding av kategoriska variabler

In [121]:
x_encoded.dtypes  # Visar datatyperna efter one-hot encoding
x_encoded.shape  # Visar antalet rader och kolumner efter one-hot encoding

(562, 14)

In [122]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(
    x_encoded, y, test_size=0.2, random_state=42
) # Här delar upp data i tränings- och testset (80% träning, 20% test)

In [123]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()  # Här skapar en instans av LinearRegression-modellen
model.fit(x_train, y_train)  # Här tränar modellen på träningsdata

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [124]:
y_pred = model.predict(x_test)  # Gör prediktioner på testdata

In [125]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

mae = mean_absolute_error(y_test, y_pred)            # Beräknar Mean Absolute Error
rmse = np.sqrt(mean_squared_error(y_test, y_pred))   # Beräknar Root Mean Squared Error
r2 = r2_score(y_test, y_pred)                        # Beräknar R-squared värde

mae, rmse, r2   # Visar utvärderingsmåtten för modellen


(9.485349730081861, np.float64(17.029159908497974), 0.8845016033484159)

In [126]:
coefficients = pd.Series(
    model.coef_,
    index=x_train.columns
).sort_values(key=abs, ascending=False)

coefficients # Jag analyserar modellens koefficienter för att si vilka faktorer som har störst påverkan på priset
# En lista av vilka variabler som påverkar priset mest, om de höjer eller sänker priset och med hur mycket

Per_Minute_Rate              60.672855
Per_Km_Rate                  25.344057
Traffic_Conditions_Low       -4.445463
Traffic_Conditions_Medium    -4.337183
Time_of_Day_Morning           2.065111
Time_of_Day_Evening          -1.940393
Day_of_Week_Weekend           1.818457
Trip_Distance_km              1.747364
Weather_Rain                 -1.409732
Weather_Snow                  1.212143
Time_of_Day_Night            -0.674383
Base_Fare                    -0.509390
Passenger_Count              -0.383629
Trip_Duration_Minutes         0.299364
dtype: float64

In [127]:
import joblib

joblib.dump(model, "linear_regression_model.pkl")  # Sparar den tränade modellen till en fil
joblib.dump(x_train.columns, "model_features.pkl")  # Sparar funktionernas kolumnnamn till en fil

['model_features.pkl']

In [128]:
example = pd.DataFrame([{
    "Trip_Distance_km": 10,         # Exempelvärde för sträcka i kilometer
    "Passenger_Count": 2,           # Exempelvärde för antal passagerare
    "Base_Fare": 3,                 # Exempelvärde för grundavgift
    "Per_Km_Rate": 1.2,             # Exempelvärde för pris per kilometer
    "Per_Minute_Rate": 0.3,         # Exempelvärde för pris per minut
    "Trip_Duration_Minutes": 20,    # Exempelvärde för resans varaktighet i minuter
    "Time_of_Day_Afternoon": 1,     # Exempelvärde för tid på dagen (1 om eftermiddag, annars 0)
    "Day_of_Week_Weekend": 0,       # Exempelvärde för veckodag (1 om helg, annars 0)
    "Traffic_Conditions_Medium": 1, # Exempelvärde för trafikförhållanden (1 om medium, annars 0)
    "Weather_Clear": 1              # Exempelvärde för väderförhållanden (1 om klart väder, annars 0)
}])

# Säkerställer att kolumnerna matchar modellens förväntade funktioner
example = example.reindex(columns=x_train.columns, fill_value=0) 

model.predict(example) # Gör en prediktion med hjälp av det exempeldata som skapats

predicted_price = model.predict(example)[0]         # Extraherar det predikterade priset från modellens output
print(f"Predikterat pris: {predicted_price:.2f}")   # Skriver ut det predikterade priset med två decimaler


Predikterat pris: 13.35
